In [2]:
print("Hello World")

Hello World


In [5]:
import base64
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def image_to_text(image_path: str) -> str:
    with open(image_path, "rb") as img:
        image_base64 = base64.b64encode(img.read()).decode("utf-8")

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "system",
                "content": "You are an OCR engine. Extract all visible text exactly as it appears. Preserve Nepali Unicode. Do not summarize."
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Extract all text from this image."},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{image_base64}"
                        }
                    }
                ]
            }
        ],
        temperature=0
    )

    return response.choices[0].message.content


In [11]:
# pip install python-multipart

In [12]:
from fastapi import FastAPI, UploadFile, File
import shutil
from ocr import image_to_text

app = FastAPI(title="AI OCR App")

@app.post("/extract-text")
async def extract_text(file: UploadFile = File(...)):
    image_path = f"temp_{file.filename}"

    with open(image_path, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    extracted_text = image_to_text(image_path)

    return {
        "filename": file.filename,
        "text": extracted_text
    }


In [ ]:
# uvicorn main:app --reload


SyntaxError: invalid syntax (907613299.py, line 1)